#### OK - 1 - Testar com base de dados de Iris (ver se a predição deu certo) 
#### OK - 2 - Comparar com o algorítmo de KNN real
#### 3 - Melhorar nomes das variáveis
#### 4 - Comentar funções
#### OK - 5 - Revisar funções em private OO
#### OK - 6 - Criar função de score
#### 7 - Revisar de há algum função proibida
#### 8 - Criar novas funções de cálculo de distância (Euclidiana, Manhattan, Minkowski e distância de Hamming. As primeiras três funções são usadas para a função contínua e a quarta (Hamming) para variáveis categóricas)
#### 9 - Criar Matriz de Confusão
#### 10 - Finalizar nootebook (instânciando o meu modelo e comparando com o modelo original KNN)

In [437]:
# Observações quanto ao exercício

# Funções atômicas para facilitar os testes unitários

In [438]:
import math # Para calcular a raiz quadrada
import csv # Para abrir o arquivo CSV e carregar na memória
import random # Para gerar a aleatoriedade para separar as bases de teste e train

In [464]:
class ManupulaArquivo:
    '''
    Esta classe é responsável por fornecer métodos que permitam a manipulação de arquivos para trabalhar com
    bases de dados, especificamente, para trabalhar com algorítmos de machine learning (ML).
    
    Lista de Atributos:
        N/A
    
    Lista de Métodos Públicos:
        load_dataset = carrega um dataset na memoria.
        train_test_split = separa um dataset em duas listas (variáveis repostas e variáveis preditoras) de treino
                           e duas listas (variáveis repostas e variáveis preditoras) de teste, utilizando o método
                           de validação cruzada holdout.
    
    Lista de Métodos Privados:
        __shuffle_dataset = randomiza um dataset.
        __separate_dataset = separa o dataset em duas listas de treino e teste.
        __remove_predictor_element = remove o elemento preditor das lista de treino e teste.
    '''
    
    def load_dataset(self, path_file):
        '''
        Método público que carrega um csv na memória do nootbook.
        Retorna a base de dados em formato de lista.
        
        Args:
            path_file(str): O caminho do arquivo CSV. Caso esteja na mesma pasta do nootbook, basta inserir o
                            nome do arquivo. Não é necessário adicionar a extensão do arquivo (.csv).
        
        Returns:
            dataset(list): Uma lista com os dados do CSV de entrada carregado na memória.
        '''
        with open(path_file) as csv_file:
            dataset = list(csv.reader(csv_file))
        return dataset

    def __shuffle_dataset(self, dataset, test_ratio):
        '''
        Método privado que embaralha o dataset, isto é, randomiza a base de dados.
        Retorna a base de dados randomizada e a proporção da base para teste do modelo.
        
        Args:
            dataset(list): A base de dados contendo os dados em formato de lista carregados na memória.
            
            test_ratio(float): Tamanho (proporção) da base de teste em valor percentual.
            
        Returns:
            shuffled_dataset(list): Uma lista com os dados do dataset randomizados (embaralhados).
            
            test_size(int): Tamanho (inteiro) da base de dados em valor absoluto (Ex: 200 linhas do dataset randomizado). 
        '''
        shuffled_dataset = random.sample(dataset, len(dataset))
        test_size = int(len(dataset) * test_ratio)        
        return shuffled_dataset, test_size 
    
    def __separate_dataset(self, shuffled_dataset, test_size):
        '''
        Método privado que separa um dataset, preferencialmente randomizado (embaralhados), em duas lista: uma para treino
        e outra para teste.
        Retorna as duas listas de treino e teste.
        
        Args:
            shuffled_dataset(list): Uma lista com os dados do dataset, preferencialmente randomizados (embaralhados).
            
            test_size(float): Tamanho (proporção) da base de teste.
            
        Returns:
            train_data(list): Uma lista com os dados para treinamento do modelo, de acordo com a proporção
            fornecida de teste.
            
            test_data(list): Uma lista com os dados para teste do modelo, de acordo com a proporção
            fornecida de teste.
        '''
        test_data = shuffled_dataset[:test_size]
        train_data = shuffled_dataset[test_size:]
        return train_data, test_data
    
    def __remove_predictor_element(self, train_data, test_data):
        '''
        Método privado que remove o elemento preditor, considerando como sendo o último elemento, do dataset.
        Retorna quatro lista contendo os elementos seletores (varíaveis respostas) e os valores preditores
        de treino e teste.
        
        Args:
            train_data(list): Uma lista com os dados para treinamento do modelo, de acordo com a proporção
            fornecida de teste.
            
            test_data(list): Uma lista com os dados para teste do modelo, de acordo com a proporção
            fornecida de teste.
        
        Returns:
            train_X(list): Uma lista contendo os elementos seletores (varíaveis respostas) da lista de treino.
            
            train_y(list): Uma lista contendo os elementos seletores (varíaveis respostas) da lista de teste.
            
            test_X(list): Uma lista contendo os elementos preditores (varíaveis preditoras) da lista de treino.
            
            test_y(list): Uma lista contendo os elementos preditores (varíaveis preditoras) da lista de teste.
        '''
        last_element = -1
        train_X = [sample[:last_element] for sample in train_data]
        train_y = [sample[last_element] for sample in train_data]
        test_X = [sample[:last_element] for sample in test_data]
        test_y = [sample[last_element] for sample in test_data]
        return train_X, train_y, test_X, test_y
    
    def train_test_split(self, dataset, test_ratio=0.3):
        '''
        Método público que separa uma base de dados em quatro listas para treino e teste de um modelo de machine learning
        utilizando o método de validação cruzada holdout.
        Retorna quatro lista contendo os elementos seletores (varíaveis respostas) e os valores preditores
        de treino e teste.
        
        Args:
            dataset(list): A base de dados contendo os dados em formato de lista carregados na memória.
            
            test_ratio(float): Tamanho (proporção) da base de teste em valor percentual. Por default o valor
            desta variável é 0.3 (30%).
        
        Returns:
            train_X(list): Uma lista contendo os elementos seletores (varíaveis respostas) da lista de treino.
            
            train_y(list): Uma lista contendo os elementos seletores (varíaveis respostas) da lista de teste.
            
            test_X(list): Uma lista contendo os elementos preditores (varíaveis preditoras) da lista de treino.
            
            test_y(list): Uma lista contendo os elementos preditores (varíaveis preditoras) da lista de teste.
        
        '''
        shuffled_dataset, test_size = self.__shuffle_dataset(dataset, test_ratio)
        train_data, test_data = self.__separate_dataset(shuffled_dataset, test_size)
        train_X, train_y, test_X, test_y = self.__remove_predictor_element(train_data, test_data)
        return train_X, train_y, test_X, test_y    

In [465]:
# Classe KNN
class KNeighborsClassifier:
    '''
        
    '''

    def __init__(self, k=3, method='euclidean'):
        '''
        
        '''
        self.k = k
        self.method = method
    
    def fit(self, x_train, y_train):
        '''
        
        '''
        self.x_train = x_train
        self.y_train = y_train
    
    def calculate_distance(self, points_p, points_q):
        '''
        
        '''
        sum_points = 0
        
        # De acordo com o Wikipedia, a distância euclidiana é dado pela Raiz Quadrada do Somatório((Pi - Qi)²)
        if self.method == 'euclidean': 
            dimension_points = len(points_p)
            #print('dimension_points: ', dimension_points)
            #print(points_p,"//", points_q, "\n\n")
            for i in range(dimension_points):
                #print(points_p[i], points_q[i], float(points_p[i]), float(points_q[i]))
                sum_points += (float(points_p[i]) - float(points_q[i]))**2
                #print(sum_points)
        
        # De acordo com ....
        elif self.method == 'manhattan':
            sum_points = 9
            
        return math.sqrt(sum_points)
    
    def compara_distancias(self, instancia_test):
        '''
        
        '''
        result_compara = []
        dimension_x_train = len(self.x_train)
        for i in range(dimension_x_train):
            result_compara.append(self.calculate_distance(self.x_train[i], instancia_test))
        return result_compara
    
    def encontra_menor_distancia(self, result_compara):
        '''
        
        '''
        ranged = range(len(result_compara))
        return sorted(ranged, key=lambda i: result_compara[i])[:self.k]
    
    def encontra_label(self, k_menores_distancias):
        '''
        
        '''
        labels_k = []
        for i in k_menores_distancias:
            labels_k.append(self.y_train[i])
        return labels_k
    
    def predict_label(self, labels_k):
        '''
        
        '''
        return max(labels_k, key=labels_k.count)

    def predict(self, x_test):
        '''
        
        '''
        result_predict = []
        #print('x_test: ', x_test)
        for instance_x_test in x_test:
            #print('instância: ', instance_x_test)
            result_distance = self.compara_distancias(instance_x_test)
            result_menor_distance = self.encontra_menor_distancia(result_distance)
            result_label = self.encontra_label(result_menor_distance)
            result_predict_label = self.predict_label(result_label)
            result_predict.append(result_predict_label)                
        return result_predict
    
    def prediction_report(self, predictions, test_y):
        '''
        
        '''
        for i, prediction in enumerate(predictions):
            print('Real:', test_y[i], '\t Predito:', prediction)

    def accuracy_report(self, predictions, test_y):
        '''
        
        '''
        acertos = 0
        for i, j in zip(predictions, test_y):
            if i == j:
                acertos += 1
                
        #accuracy = sum(i == j for i, j in zip(predictions, test_y)) / len(test_y) * 100.0
        print('Quantidade de Treinos:', len(self.x_train))
        print('Quantidade de Testes:', len(test_y))
        print('Quantidade de Acertos:', acertos)
        print('Quantidade de Erros:', (len(test_y)-acertos))
        print('Acurácia do Modelo: %.2f%%' % (acertos*100/len(test_y)))
        
        # Matriz de Confusão

In [474]:
ma = ManupulaArquivo()
dataset = ma.load_dataset("iris.data")
train_X, train_y, test_X, test_y = ma.train_test_split(dataset, test_ratio=0.2)
print('\n',len(train_X), len(train_y), len(test_X), len(test_y))
print(train_X[:3], '\n', train_y[:3], '\n', test_X[:3], '\n', test_y[:3])


 120 120 30 30
[['5.2', '2.7', '3.9', '1.4'], ['5.1', '3.5', '1.4', '0.2'], ['5.6', '2.8', '4.9', '2.0']] 
 ['Iris-versicolor', 'Iris-setosa', 'Iris-virginica'] 
 [['6.9', '3.2', '5.7', '2.3'], ['5.5', '3.5', '1.3', '0.2'], ['6.3', '2.3', '4.4', '1.3']] 
 ['Iris-virginica', 'Iris-setosa', 'Iris-versicolor']


In [475]:
knn2 = KNeighborsClassifier()
knn2.fit(train_X,train_y)
print(len(train_X))

120


In [476]:
result = knn2.predict(test_X)

In [477]:
knn2.accuracy_report(result, test_y)

Quantidade de Treinos: 120
Quantidade de Testes: 30
Quantidade de Acertos: 28
Quantidade de Erros: 2
Acurácia do Modelo: 93.33%


In [478]:
knn2.prediction_report(result, test_y)

Real: Iris-virginica 	 Predito: Iris-virginica
Real: Iris-setosa 	 Predito: Iris-setosa
Real: Iris-versicolor 	 Predito: Iris-versicolor
Real: Iris-versicolor 	 Predito: Iris-versicolor
Real: Iris-versicolor 	 Predito: Iris-versicolor
Real: Iris-setosa 	 Predito: Iris-setosa
Real: Iris-virginica 	 Predito: Iris-virginica
Real: Iris-setosa 	 Predito: Iris-setosa
Real: Iris-versicolor 	 Predito: Iris-versicolor
Real: Iris-setosa 	 Predito: Iris-setosa
Real: Iris-versicolor 	 Predito: Iris-versicolor
Real: Iris-setosa 	 Predito: Iris-setosa
Real: Iris-setosa 	 Predito: Iris-setosa
Real: Iris-setosa 	 Predito: Iris-setosa
Real: Iris-versicolor 	 Predito: Iris-virginica
Real: Iris-setosa 	 Predito: Iris-setosa
Real: Iris-versicolor 	 Predito: Iris-versicolor
Real: Iris-setosa 	 Predito: Iris-setosa
Real: Iris-virginica 	 Predito: Iris-virginica
Real: Iris-versicolor 	 Predito: Iris-versicolor
Real: Iris-versicolor 	 Predito: Iris-versicolor
Real: Iris-setosa 	 Predito: Iris-setosa
Real: Iri

In [479]:
# Imports necessários
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np

In [480]:
dataset = load_iris()
X_train1, X_test1, Y_train1, Y_test1 = train_test_split(dataset['data'], dataset['target'], random_state=0)
print(X_train1.shape)
print(X_test1.shape)

(112, 4)
(38, 4)


In [481]:
# Instanciando o modelo KNN
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors = 3)

# Treinando os dados
knn.fit(train_X,train_y)

# Para medir a accuracy, utiliza-se o método score passando a base de teste e os valores preditos
knn.score(test_X, test_y)

0.9333333333333333

In [356]:
def load_dataseta(path_file):
    with open(path_file) as csv_file:
        dataset = list(csv.reader(csv_file))
    return dataset

In [202]:
t = load_dataseta("iris.data")
t

NameError: name 'csvfile' is not defined

In [200]:
print(ma.load_dataset.__doc__)


        Função que carrega um csv na memória do nootbook.
        Returna a base de dados em formato de lista
        
        Args:
            path_file(str): O caminho do arquivo CSV. Caso esteja na mesma pasta do nootbook, basta inserir o
                            nome do arquivo. Não é necessário adicionar a extensão do arquivo (.csv).
        
        Returns:
            dataset(list): Uma lista com os dados do CSV de entrada carregado na memória.
        
        


In [ ]:
ma.